**Disclaimer:** The distributed notebook only serves for demonstration purposes. Jupyter notebooks are not recommended for preCICE development or deployment. Feel free to convert this notebook to a plain python script for improved error messages using `jupyter nbconvert --to python Solver*.ipynb`

In [ ]:
def dummySolidSolver(x, y, dt):
    return x + dt * y**2

In [ ]:
import precice
#precice_config = "precice-config-both-init.xml"
precice_config = "precice-config-experimental-both-init.xml"
solverInterface = precice.Interface("Solid", precice_config, 0,1)
meshId = solverInterface.get_mesh_id("Solid-Mesh")
vertexId = solverInterface.set_mesh_vertex(meshId, [1,2,3])
readDataId = solverInterface.get_data_id("Force", meshId)
writeDataId = solverInterface.get_data_id("Displacement", meshId)

In [ ]:
precice_dt = solverInterface.initialize()

In [ ]:
window_size = precice_dt
solver_dt = window_size/3
time = 0

### Initialize data
optional, if `initialize = "false"` or not given

In [ ]:
initDisplacement = 2
writeData = initDisplacement
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
solverInterface.mark_action_fulfilled(precice.action_write_initial_data())

In [ ]:
solverInterface.initialize_data()

### First window, first iteration

require to write checkpoint, when entering window:

In [ ]:
solverInterface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())

#### First subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))

In [ ]:
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

#### Second subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

#### Third subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

In [ ]:
solverInterface.is_time_window_complete()

### First window, second iteration
require to read checkpoint, when repeating window:

In [ ]:
solverInterface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())

#### First subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))

In [ ]:
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))

In [ ]:
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

#### Second subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

#### Third subcycle

In [ ]:
readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
print("readData = {}".format(readData))
print("Start time: {}".format(time))
writeData = dummySolidSolver(writeData, readData, solver_dt)
time += solver_dt
print("End time: {}".format(time))
print("writeData = {}".format(writeData))
solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
precice_dt = solverInterface.advance(solver_dt)
print("precice_dt = {}".format(precice_dt))

In [ ]:
solverInterface.is_time_window_complete()

### Second window:
... same procedure as before.

In [ ]:
window = 2
iteration = 1

while solverInterface.is_coupling_ongoing():
    if solverInterface.is_action_required(precice.action_write_iteration_checkpoint()):
        solverInterface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())
        
    readData = solverInterface.read_scalar_data(readDataId, vertexId, solver_dt)
    print("readData = {}".format(readData))
    
    writeData = dummySolidSolver(writeData, readData, solver_dt)
    
    print("writeData = {}".format(writeData))
    solverInterface.write_scalar_data(writeDataId, vertexId, writeData)
    
    precice_dt = solverInterface.advance(solver_dt)
    print("precice_dt = {}".format(precice_dt))
    
    if solverInterface.is_time_window_complete():
        window += 1
        iteration = 1
    if solverInterface.is_action_required(precice.action_read_iteration_checkpoint()):
        iteration += 1
    
    if solverInterface.is_coupling_ongoing():
        print("window = {}".format(window))
        print("iteration = {}".format(iteration))
        
    if solverInterface.is_action_required(precice.action_read_iteration_checkpoint()):
        solverInterface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())
        
print("Done!")
solverInterface.finalize()